# Zadanie 3 (7 punktów)

Celem ćwiczenia jest imlementacja metody [Minimax z obcinaniem alpha-beta](https://en.wikipedia.org/wiki/Alpha%E2%80%93beta_pruning) do gry  Czwórki (ang. Connect Four).

W trakcie ćwiczenia można skorzystać z reposytorium z implementacją gry [Connect Four udostępnionym przez Jakuba Łyskawę](https://github.com/lychanl/two-player-games). Ewentualnie, można zaimplementować samemu grę Connect Four.

Należy zaimplementować co najmniej dwie heurystyki do ewaluacji planszy.  

Implementację algorytmu Minimax (klasa `MiniMaxSolver`) należy przetestować używając różną głębokość przeszukiwania symulując grę "komputer vs komputer". W eksperymentach należy również zademonstrować różnice pomiędzy heurystykami.  

W ramach zadania można zaimplementować dowolną liczbę dodatkowych metod w klasie `MiniMaxSolver`.

Punktacja:

- Działająca metoda Minimax oraz heurystyki do ewaluacji planszy. - **2 pkt**
- Działająca metoda Minimax z obcinaniem alpha-beta. - **1.5 pkt**
- Analiza jakości solvera w zależności od głębokości przeszukiwania - wykresy. - **2pkt**
    - należy zaimplementować w tym celu prostą wizualizację rozgrywki dwóch agentów
- Jakość kodu. - **1.5pkt**

Aby importowanie elementów z poniższej komórki działało należy umieścić tego notebooka w tym samym folderze co paczkę `two_player_games`:
```
├── LICENSE
├── README.md
├── minimax.ipynb # <<< HERE
├── test
│   ├── __init__.py
│   ├── test_connect_four.py
│   ├── test_dots_and_boxes.py
│   └── test_pick.py
└── two_player_games
    ├── __init__.py
    ├── games
    │   ├── connect_four.py
    │   └── dots_and_boxes.py
    ├── move.py
    ├── player.py
    └── state.py
```

In [ ]:
from typing import Tuple, List, Callable, Any

from game.two_player_games.player import Player
from game.two_player_games.games.connect_four import ConnectFour, ConnectFourMove

ModuleNotFoundError: No module named 'two_player_games'

Wielkość planszy

In [3]:
ROW_COUNT = 6
COLUMN_COUNT = 7

In [ ]:
class Heuristic:
    def h(game: ConnectFour):
        game_state = game.state
        curr_player = game_state.get_current_player()

        pass

In [ ]:
class MinMaxSolver:

    def __init__(self, game: ConnectFour, heuristic: Callable[[ConnectFour, Player], Any]):
        self.game = game
        self.heuristic = heuristic

    def get_best_move(self)->int:
        self.game.get_moves()

    def is_valid_move(self, col_index:int)->bool:
        pass

    def minimax(self, depth, alpha:float, beta:float, is_maximizing_player:bool)-> Tuple[int, float]:
        """Returns column index and score"""

        # FAIL-HARD PSEUDOCODE

        # if depth == 0 or node is terminal
        # then return heuristic value of the node

        if depth == 0 or self.game.is_finished():
            return self.heuristic(self.game)

        # if maximizing_player move then

        #     value is minus infinity

        #     for each child of node do
        #         value := max(value. minimax(child, depth -1, alpha, beta, is_maximizing=FALSE))

        #         if value > beta then
        #             break

        #         alpha := max(alpha, value)

        #     return value

        # else if minimizing_player move then

        #     value is plus infinity

        #     for each child of node do
        #         value := min(value. minimax(child, depth -1, alpha, beta, is_maximizing=TRUE))

        #         if value < alpha then
        #             break

        #         beta := min(beta, value)

        #     return value

        if is_maximizing_player:

            value = float("-inf");

            for move in self.game.get_moves():
                value = min(value, self.minimax(move, depth - 1, alpha, beta, False))

                if value > beta:
                    break

                alpha = max(alpha, value)

            return value

        else:

            value = float("inf");

            for move in self.game.get_moves():
                value = min(value, self.minimax(move, depth - 1, alpha, beta, True))

                if value < alpha:
                    break;

                beta = min(beta, value)

            return value
        pass

Rozgrywka

In [5]:
p1 = Player("a")
p2 = Player("b")
game = ConnectFour(size=(COLUMN_COUNT, ROW_COUNT), first_player=p1, second_player=p2)
game.make_move(ConnectFourMove(3))
game.make_move(ConnectFourMove(4))
game.make_move(ConnectFourMove(3))

print(game)

Current player: b
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][a][ ][ ][ ]
[ ][ ][ ][a][b][ ][ ]
